In [ ]:
import numpy as np
import pandas as pd 
from sklearn.metrics import (confusion_matrix, precision_recall_curve, average_precision_score,
                             roc_auc_score, classification_report, f1_score)
import joblib
import os
os.chdir(r'C:/Users/Marc/Desktop/TFG/R Files/Anomaly Detection for SPC')
from Src.FeatureExt.ML_AD_Preprocessing import *
from sklearn.preprocessing import StandardScaler
from plotly import graph_objects as go
from plotly.subplots import make_subplots

#import data and ML Modelss
TS1 = pd.read_csv("Data/TimeSeries1.csv", index_col = 0)
TS1 = TS1.iloc[1280:1600, :]
TS1_Class = pd.read_csv("Data/TimeSeries1_Classification.csv", index_col = 0)
TS1_Class = TS1_Class.iloc[1280:1600, 19:60]
TS1_Class = np.array(TS1_Class).reshape(TS1_Class.size)

TS1_WE_Class = pd.read_csv("Data/TimeSeries1_WE_Classification.csv", index_col = 0)
TS1_WE_Class = np.array(TS1_WE_Class.iloc[1280:1600])\
    .reshape(TS1_WE_Class.iloc[1280:1600].size)

TS1_Nelson_Class = pd.read_csv("Data/TimeSeries1_Nelson_Classification.csv", index_col = 0)
TS1_Nelson_Class = np.array(TS1_Nelson_Class.iloc[1280:1600])\
    .reshape(TS1_Nelson_Class.iloc[1280:1600].size)

iforest = joblib.load('ML_Models/ML_iforest.sav')

SVM = joblib.load('ML_Models/ML_SVM.sav')

In [ ]:
#Pre-processing
#Standardize data based on first 20 points of chart, which is supposed in control       
X_train = stdvector(TS1)

#Extract features
X_test = sw_dataset_3(X_train, 20)
X_test = np.transpose(X_test)
scaler = StandardScaler()
scaler = scaler.fit(X_test)
X_test = pd.DataFrame(X_test,\
     columns = ["last_value", "mean20", "sigma20","mean5", "sigma5", "find_if", "kurtosis",
     "dir_change", 'wavg', 'slope', 'meancross', 'rdist', 'brange'])
X_test_SVM = scaler.transform(X_test)
X_test_SVM = pd.DataFrame(X_test_SVM, columns = ["last_value", "mean20", "sigma20","mean5", "sigma5", "find_if", "kurtosis",
     "dir_change", 'wavg', 'slope', 'meancross', 'rdist', 'brange'])



In [ ]:
#predict from ML Modelss
#Predict iForest
predictions_forest = iforest.predict(np.array(X_test))
predictions_forest = np.array((predictions_forest == -1)*1)


In [ ]:
#Predict SVM
predictions_SVM = SVM.predict(np.array(X_test_SVM))

In [ ]:
#Confusion Matrices Global
cm_WE = confusion_matrix(TS1_Class, TS1_WE_Class)
cm_Nelson = confusion_matrix(TS1_Class, TS1_Nelson_Class)
cm_forest = confusion_matrix(TS1_Class, predictions_forest)
cm_SVM = confusion_matrix(TS1_Class, predictions_SVM)

#Confusion Matrices per error type
TS1_Class_Normal = TS1_Class[0:1640]
TS1_Class_Cyclic = TS1_Class[1640:3280]
TS1_Class_Systematic = TS1_Class[3280:4920]
TS1_Class_Stratified = TS1_Class[4920:6560]
TS1_Class_ut = TS1_Class[6560:8200]
TS1_Class_dt = TS1_Class[8200:9840]
TS1_Class_us = TS1_Class[9840:11480]
TS1_Class_ds = TS1_Class[11480:13120]

TS1_WE_Class_Normal = TS1_WE_Class[0:1640]
TS1_WE_Class_Cyclic = TS1_WE_Class[1640:3280]
TS1_WE_Class_Systematic = TS1_WE_Class[3280:4920]
TS1_WE_Class_Stratified = TS1_WE_Class[4920:6560]
TS1_WE_Class_ut = TS1_WE_Class[6560:8200]
TS1_WE_Class_dt = TS1_WE_Class[8200:9840]
TS1_WE_Class_us = TS1_WE_Class[9840:11480]
TS1_WE_Class_ds = TS1_WE_Class[11480:13120]

TS1_Nelson_Class_Normal = TS1_Nelson_Class[0:1640]
TS1_Nelson_Class_Cyclic = TS1_Nelson_Class[1640:3280]
TS1_Nelson_Class_Systematic = TS1_Nelson_Class[3280:4920]
TS1_Nelson_Class_Stratified = TS1_Nelson_Class[4920:6560]
TS1_Nelson_Class_ut = TS1_Nelson_Class[6560:8200]
TS1_Nelson_Class_dt = TS1_Nelson_Class[8200:9840]
TS1_Nelson_Class_us = TS1_Nelson_Class[9840:11480]
TS1_Nelson_Class_ds = TS1_Nelson_Class[11480:13120]


In [ ]:
predictions_forest_Normal = predictions_forest[0:1640]
predictions_forest_Cyclic = predictions_forest[1640:3280]
predictions_forest_Systematic = predictions_forest[3280:4920]
predictions_forest_Stratified = predictions_forest[4920:6560]
predictions_forest_ut = predictions_forest[6560:8200]
predictions_forest_dt = predictions_forest[8200:9840]
predictions_forest_us = predictions_forest[9840:11480]
predictions_forest_ds = predictions_forest[11480:13120]

predictions_SVM_Normal = predictions_SVM[0:1640]
predictions_SVM_Cyclic = predictions_SVM[1640:3280]
predictions_SVM_Systematic = predictions_SVM[3280:4920]
predictions_SVM_Stratified = predictions_SVM[4920:6560]
predictions_SVM_ut = predictions_SVM[6560:8200]
predictions_SVM_dt = predictions_SVM[8200:9840]
predictions_SVM_us = predictions_SVM[9840:11480]
predictions_SVM_ds = predictions_SVM[11480:13120]

cm_perfect_normal = confusion_matrix(TS1_Class_Normal, TS1_Class_Normal)
cm_perfect_Cyclic = confusion_matrix(TS1_Class_Cyclic, TS1_Class_Cyclic)
cm_perfect_Systematic = confusion_matrix(TS1_Class_Systematic, TS1_Class_Systematic)
cm_perfect_Stratified = confusion_matrix(TS1_Class_Stratified, TS1_Class_Stratified)
cm_perfect_ut = confusion_matrix(TS1_Class_ut, TS1_Class_ut)
cm_perfect_dt = confusion_matrix(TS1_Class_dt, TS1_Class_dt)
cm_perfect_us = confusion_matrix(TS1_Class_us, TS1_Class_us)
cm_perfect_ds = confusion_matrix(TS1_Class_ds, TS1_Class_ds)

cm_WE_normal = confusion_matrix(TS1_Class_Normal, TS1_WE_Class_Normal)
cm_WE_Cyclic = confusion_matrix(TS1_Class_Cyclic, TS1_WE_Class_Cyclic)
cm_WE_Systematic = confusion_matrix(TS1_Class_Systematic, TS1_WE_Class_Systematic)
cm_WE_Stratified = confusion_matrix(TS1_Class_Stratified, TS1_WE_Class_Stratified)
cm_WE_ut = confusion_matrix(TS1_Class_ut, TS1_WE_Class_ut)
cm_WE_dt = confusion_matrix(TS1_Class_dt, TS1_WE_Class_dt)
cm_WE_us = confusion_matrix(TS1_Class_us, TS1_WE_Class_us)
cm_WE_ds = confusion_matrix(TS1_Class_ds, TS1_WE_Class_ds)

cm_Nelson_normal = confusion_matrix(TS1_Class_Normal, TS1_Nelson_Class_Normal)
cm_Nelson_Cyclic = confusion_matrix(TS1_Class_Cyclic, TS1_Nelson_Class_Cyclic)
cm_Nelson_Systematic = confusion_matrix(TS1_Class_Systematic, TS1_Nelson_Class_Systematic)
cm_Nelson_Stratified = confusion_matrix(TS1_Class_Stratified, TS1_Nelson_Class_Stratified)
cm_Nelson_ut = confusion_matrix(TS1_Class_ut, TS1_Nelson_Class_ut)
cm_Nelson_dt = confusion_matrix(TS1_Class_dt, TS1_Nelson_Class_dt)
cm_Nelson_us = confusion_matrix(TS1_Class_us, TS1_Nelson_Class_us)
cm_Nelson_ds = confusion_matrix(TS1_Class_ds, TS1_Nelson_Class_ds)

cm_forest_normal = confusion_matrix(TS1_Class_Normal, predictions_forest_Normal)
cm_forest_Cyclic = confusion_matrix(TS1_Class_Cyclic, predictions_forest_Cyclic)
cm_forest_Systematic = confusion_matrix(TS1_Class_Systematic, predictions_forest_Systematic)
cm_forest_Stratified = confusion_matrix(TS1_Class_Stratified, predictions_forest_Stratified)
cm_forest_ut = confusion_matrix(TS1_Class_ut, predictions_forest_ut)
cm_forest_dt = confusion_matrix(TS1_Class_dt, predictions_forest_dt)
cm_forest_us = confusion_matrix(TS1_Class_us, predictions_forest_us)
cm_forest_ds = confusion_matrix(TS1_Class_ds, predictions_forest_ds)

cm_SVM_normal = confusion_matrix(TS1_Class_Normal, predictions_SVM_Normal)
cm_SVM_Cyclic = confusion_matrix(TS1_Class_Cyclic, predictions_SVM_Cyclic)
cm_SVM_Systematic = confusion_matrix(TS1_Class_Systematic, predictions_SVM_Systematic)
cm_SVM_Stratified = confusion_matrix(TS1_Class_Stratified, predictions_SVM_Stratified)
cm_SVM_ut = confusion_matrix(TS1_Class_ut, predictions_SVM_ut)
cm_SVM_dt = confusion_matrix(TS1_Class_dt, predictions_SVM_dt)
cm_SVM_us = confusion_matrix(TS1_Class_us, predictions_SVM_us)
cm_SVM_ds = confusion_matrix(TS1_Class_ds, predictions_SVM_ds)

#Classification Report Global
cf_WE = classification_report(TS1_Class, TS1_WE_Class)
cf_Nelson = classification_report(TS1_Class, TS1_Nelson_Class)
cf_forest = classification_report(TS1_Class, predictions_forest)
cf_SVM = classification_report(TS1_Class, predictions_SVM)

#Classification Report per Error
cr_WE_normal = classification_report(TS1_Class_Normal, TS1_WE_Class_Normal)
cr_WE_Cyclic = classification_report(TS1_Class_Cyclic, TS1_WE_Class_Cyclic)
cr_WE_Systematic = classification_report(TS1_Class_Systematic, TS1_WE_Class_Systematic)
cr_WE_Stratified = classification_report(TS1_Class_Stratified, TS1_WE_Class_Stratified)
cr_WE_ut = classification_report(TS1_Class_ut, TS1_WE_Class_ut)
cr_WE_dt = classification_report(TS1_Class_dt, TS1_WE_Class_dt)
cr_WE_us = classification_report(TS1_Class_us, TS1_WE_Class_us)
cr_WE_ds = classification_report(TS1_Class_ds, TS1_WE_Class_ds)

cr_Nelson_normal = classification_report(TS1_Class_Normal, TS1_Nelson_Class_Normal)
cr_Nelson_Cyclic = classification_report(TS1_Class_Cyclic, TS1_Nelson_Class_Cyclic)
cr_Nelson_Systematic = classification_report(TS1_Class_Systematic, TS1_Nelson_Class_Systematic)
cr_Nelson_Stratified = classification_report(TS1_Class_Stratified, TS1_Nelson_Class_Stratified)
cr_Nelson_ut = classification_report(TS1_Class_ut, TS1_Nelson_Class_ut)
cr_Nelson_dt = classification_report(TS1_Class_dt, TS1_Nelson_Class_dt)
cr_Nelson_us = classification_report(TS1_Class_us, TS1_Nelson_Class_us)
cr_Nelson_ds = classification_report(TS1_Class_ds, TS1_Nelson_Class_ds)

cr_forest_normal = classification_report(TS1_Class_Normal, predictions_forest_Normal)
cr_forest_Cyclic = classification_report(TS1_Class_Cyclic, predictions_forest_Cyclic)
cr_forest_Systematic = classification_report(TS1_Class_Systematic, predictions_forest_Systematic)
cr_forest_Stratified = classification_report(TS1_Class_Stratified, predictions_forest_Stratified)
cr_forest_ut = classification_report(TS1_Class_ut, predictions_forest_ut)
cr_forest_dt = classification_report(TS1_Class_dt, predictions_forest_dt)
cr_forest_us = classification_report(TS1_Class_us, predictions_forest_us)
cr_forest_ds = classification_report(TS1_Class_ds, predictions_forest_ds)

cr_SVM_normal = classification_report(TS1_Class_Normal, predictions_SVM_Normal)
cr_SVM_Cyclic = classification_report(TS1_Class_Cyclic, predictions_SVM_Cyclic)
cr_SVM_Systematic = classification_report(TS1_Class_Systematic, predictions_SVM_Systematic)
cr_SVM_Stratified = classification_report(TS1_Class_Stratified, predictions_SVM_Stratified)
cr_SVM_ut = classification_report(TS1_Class_ut, predictions_SVM_ut)
cr_SVM_dt = classification_report(TS1_Class_dt, predictions_SVM_dt)
cr_SVM_us = classification_report(TS1_Class_us, predictions_SVM_us)
cr_SVM_ds = classification_report(TS1_Class_ds, predictions_SVM_ds)

In [ ]:
cm_global = [cm_WE,cm_Nelson,cm_forest,cm_SVM]
cm_normal=[cm_WE_normal,cm_Nelson_normal,cm_forest_normal,cm_SVM_normal]
cm_Stratified=[cm_WE_Stratified,cm_Nelson_Stratified,cm_forest_Stratified,cm_SVM_Stratified]
cm_Systematic=[cm_WE_Systematic,cm_Nelson_Systematic,cm_forest_Systematic,cm_SVM_Systematic]
cm_Cyclic=[cm_WE_Cyclic,cm_Nelson_Cyclic,cm_forest_Cyclic,cm_SVM_Cyclic]
cm_ut=[cm_WE_ut,cm_Nelson_ut,cm_forest_ut,cm_SVM_ut]
cm_dt=[cm_WE_dt,cm_Nelson_dt,cm_forest_dt,cm_SVM_dt]
cm_us=[cm_WE_us,cm_Nelson_us,cm_forest_us,cm_SVM_us]
cm_ds=[cm_WE_ds,cm_Nelson_ds,cm_forest_ds,cm_SVM_ds]

In [270]:
categories=['FP','TP','FN','TN', 'FP']
names=['WE', 'Nelson', 'iForest', 'SVM']
colorsfill=['rgba(31, 119, 180,0.1)', 'rgba(255, 127, 14,0.1)',
                       'rgba(44, 160, 44,0.1)', 'rgba(214, 39, 40,0.1)',
                       'rgba(148, 103, 189,0.1)', 'rgba(140, 86, 75,0.1)',
                       'rgba(227, 119, 194,0.1)', 'rgba(127, 127, 127,0.1)']
colors=['rgba(31, 119, 180,1)', 'rgba(255, 127, 14,1)',
                       'rgba(44, 160, 44,1)', 'rgba(214, 39, 40,1)',
                       'rgba(148, 103, 189,1)', 'rgba(140, 86, 75,1)',
                       'rgba(227, 119, 194,1)', 'rgba(127, 127, 127,1)']
fig= make_subplots(rows=2, cols=4, specs=[[{'type': 'polar'}]*4]*2, subplot_titles=("Normal", "Stratified",
 "Systematic", "Cyclic", "Upward Trend", "Downward Trend", "Upward Shift", "Downward Shift"))


for i in range(len(cm_normal)):
  fig.add_trace(go.Scatterpolar(
      r=[cm_normal[i][0,1], cm_normal[i][1,1], cm_normal[i][1,0], cm_normal[i][0,0], cm_normal[i][0,1]],
      theta=categories,
      marker = dict(color=colors[i]),
      fill='toself',
      legendgroup=names[i],
      subplot = "polar1",
      name=names[i],
      fillcolor=colorsfill[i]
  ), 1, 1),
  name=names[i],
  fillcolor=colorsfill[i]
  fig.add_trace(go.Scatterpolar(
      r=[cm_Stratified[i][0,1], cm_Stratified[i][1,1], cm_Stratified[i][1,0], cm_Stratified[i][0,0], cm_Stratified[i][0,1]],
      theta=categories,
      marker = dict(color=colors[i]),
      fill='toself',
      legendgroup=names[i],
      subplot = "polar2",
      showlegend=False,
      name=names[i],
      fillcolor=colorsfill[i]
  ), 1, 2)
  fig.add_trace(go.Scatterpolar(
      r=[cm_Systematic[i][0,1], cm_Systematic[i][1,1], cm_Systematic[i][1,0], cm_Systematic[i][0,0], cm_Systematic[i][0,1]],
      theta=categories,
      marker = dict(color=colors[i]),
      fill='toself',
      legendgroup=names[i],
      subplot = "polar3",
      showlegend=False,
      name=names[i],
      fillcolor=colorsfill[i]
  ), 1, 3)
  fig.add_trace(go.Scatterpolar(
      r=[cm_Cyclic[i][0,1], cm_Cyclic[i][1,1], cm_Cyclic[i][1,0], cm_Cyclic[i][0,0], cm_Cyclic[i][0,1]],
      theta=categories,
      marker = dict(color=colors[i]),
      fill='toself',
      legendgroup=names[i],
      subplot = "polar4",
      showlegend=False,
      name=names[i],
      fillcolor=colorsfill[i]
  ), 1, 4)
  fig.add_trace(go.Scatterpolar(
      r=[cm_ut[i][0,1], cm_ut[i][1,1], cm_ut[i][1,0], cm_ut[i][0,0], cm_ut[i][0,1]],
      theta=categories,
      marker = dict(color=colors[i]),
      fill='toself',
      legendgroup=names[i],
      subplot = "polar4",
      showlegend=False,
      name=names[i],
      fillcolor=colorsfill[i]
  ), 2, 1)
  fig.add_trace(go.Scatterpolar(
      r=[cm_dt[i][0,1], cm_dt[i][1,1], cm_dt[i][1,0], cm_dt[i][0,0], cm_dt[i][0,1]],
      theta=categories,
      marker = dict(color=colors[i]),
      fill='toself',
      legendgroup=names[i],
      subplot = "polar4",
      showlegend=False,
      name=names[i],
      fillcolor=colorsfill[i]
  ), 2, 2)
  fig.add_trace(go.Scatterpolar(
      r=[cm_us[i][0,1], cm_us[i][1,1], cm_us[i][1,0], cm_us[i][0,0], cm_us[i][0,1]],
      theta=categories,
      marker = dict(color=colors[i]),
      fill='toself',
      legendgroup=names[i],
      subplot = "polar4",
      showlegend=False,
      name=names[i],
      fillcolor=colorsfill[i]
  ), 2, 3)
  fig.add_trace(go.Scatterpolar(
      r=[cm_ds[i][0,1], cm_ds[i][1,1], cm_ds[i][1,0], cm_ds[i][0,0], cm_ds[i][0,1]],
      theta=categories,
      marker = dict(color=colors[i]),
      fill='toself',
      legendgroup=names[i],
      subplot = "polar4",
      showlegend=False,
      name=names[i],
      fillcolor=colorsfill[i]
  ), 2, 4)

fig.update_layout(
    polar1=dict(
    radialaxis=dict(
    visible=True,
    range=[0, 1900]),
    angularaxis=dict(
    rotation= 45,
    direction='clockwise')
    ),
    polar2=dict(
    radialaxis=dict(
    visible=True,
    range=[0, 900]),
    angularaxis=dict(
    rotation= 45,
    direction='clockwise')
    ),
    polar3=dict(
    radialaxis=dict(
    visible=True,
    range=[0, 900]),
    angularaxis=dict(
    rotation= 45,
    direction='clockwise')
    ),
    polar4=dict(
    radialaxis=dict(
    visible=True,
    range=[0, 900]),
    angularaxis=dict(
    rotation= 45,
    direction='clockwise')
    ),
    polar5=dict(
    radialaxis=dict(
    visible=True,
    range=[0, 900]),
    angularaxis=dict(
    rotation= 45,
    direction='clockwise')
    ),
    polar6=dict(
    radialaxis=dict(
    visible=True,
    range=[0, 900]),
    angularaxis=dict(
    rotation= 45,
    direction='clockwise')
    ),
    polar7=dict(
    radialaxis=dict(
    visible=True,
    range=[0, 900]),
    angularaxis=dict(
    rotation= 45,
    direction='clockwise')
    ),
    polar8=dict(
    radialaxis=dict(
    visible=True,
    range=[0, 900]),
    angularaxis=dict(
    rotation= 45,
    direction='clockwise')
    ),
    height=700,
    width=1250,
    title = 'Classification Matrix per Error Type'
  )

fig.show()

In [266]:
fig=go.Figure()
for i in range(len(cm_global)):
    fig.add_trace(go.Scatterpolar(
        r=[cm_global[i][0,1], cm_global[i][1,1], cm_global[i][1,0], cm_global[i][0,0], cm_global[i][0,1]],
        theta=categories,
        marker = dict(color=colors[i]),
        fill='toself',
        legendgroup=names[i],
        subplot = "polar1",
        name=names[i],
        fillcolor=colorsfill[i])
    )
fig.update_layout(height=500, width=500, title='Global Classification Matrix',
polar=dict(
    radialaxis=dict(
    visible=True,
    range=[0, 7300]),
    angularaxis=dict(
    rotation= 45,
    direction='clockwise')
    ))
fig.show()

In [268]:
print(cm_forest_Cyclic)

[[702 118]
 [123 697]]


In [250]:
fig2 = go.Figure(go.Treemap(
    labels = ["Total Stratified"
    ,"Total P","Total N"
    ,'TP', 'FP', 'TN', 'FN'
    ,"WE TP","WE FP", "WE TN", "WE FN"
    ,"Nelson TP","Nelson FP", "Nelson TN", "Nelson FN"
    ,"iforest TP","iforest FP", "iforest TN", "iforest FN"
    ,"SVM3 TP","SVM3 FP", "SVM3 TN", "SVM3 FN"],
    parents = [""
    ,"Total Stratified","Total Stratified"
    ,"Total P","Total P", "Total N", "Total N"
    ,'TP', 'FP', 'TN', 'FN'
    ,'TP', 'FP', 'TN', 'FN'
    ,'TP', 'FP', 'TN', 'FN'
    ,'TP', 'FP', 'TN', 'FN'
    ,'TP', 'FP', 'TN', 'FN'],
    root_color="lightgrey",
    values=[np.sum(cm_perfect_normal)*4, 
    cm_perfect_Stratified[1,1]*4, cm_perfect_Stratified[0,0]*4,
    np.sum([cm_Stratified[0][1,1], cm_Stratified[1][1,1], cm_Stratified[2][1,1], cm_Stratified[3][1,1]]),
    np.sum([cm_Stratified[0][1,0], cm_Stratified[1][1,0], cm_Stratified[2][1,0], cm_Stratified[3][1,0]]),
    np.sum([cm_Stratified[0][0,0],  cm_Stratified[1][0,0], cm_Stratified[2][0,0], cm_Stratified[3][0,0]]),
    np.sum([cm_Stratified[0][0,1], cm_Stratified[1][0,1], cm_Stratified[2][0,1], cm_Stratified[3][0,1]]),
    cm_Stratified[0][1,1], cm_Stratified[0][1,0], cm_Stratified[0][0,0], cm_Stratified[0][0,1],
    cm_Stratified[1][1,1], cm_Stratified[1][1,0], cm_Stratified[1][0,0], cm_Stratified[1][0,1],
    cm_Stratified[2][1,1], cm_Stratified[2][1,0], cm_Stratified[2][0,0], cm_Stratified[2][0,1],
    cm_Stratified[3][1,1], cm_Stratified[3][1,0], cm_Stratified[3][0,0], cm_Stratified[3][0,1]],
    textinfo = "label+value",
    branchvalues='total'
))

fig2.update_layout(margin = dict(t=25, l=25, r=25, b=25))
fig2.show()


In [251]:
fig2 = go.Figure()
fig2.add_trace(go.Treemap(
    labels = ["Total Systematic WE"
    ,"Total Systematic Nelson"
    ,"Total Systematic iforest"
    ,"Total Systematic SVM"
    ,"Total P WE","Total N WE"
    ,"Total P Nelson","Total N Nelson"
    ,"Total P iforest","Total N iforest"
    ,"Total P SVM","Total N SVM"
    ,"WE TP","WE FP", "WE TN", "WE FN"
    ,"Nelson TP","Nelson FP", "Nelson TN", "Nelson FN"
    ,"iforest TP","iforest FP", "iforest TN", "iforest FN"
    ,"SVM TP","SVM FP", "SVM TN", "SVM FN"],
    parents = [""
    ,""
    ,""
    ,""
    ,"Total Systematic WE","Total Systematic WE"
    ,"Total Systematic Nelson","Total Systematic Nelson"
    ,"Total Systematic iforest","Total Systematic iforest"
    ,"Total Systematic SVM","Total Systematic SVM"
    ,"Total P WE","Total P WE","Total N WE","Total N WE"
    ,"Total P Nelson","Total P Nelson","Total N Nelson","Total N Nelson"
    ,"Total P iforest","Total P iforest","Total N iforest","Total N iforest"
    ,"Total P SVM","Total P SVM","Total N SVM","Total N SVM"],
    root_color="lightgrey",
    values=[np.sum(cm_perfect_normal)
    ,np.sum(cm_perfect_normal)
    ,np.sum(cm_perfect_normal)
    ,np.sum(cm_perfect_normal)
    ,cm_perfect_Systematic[1,1], cm_perfect_Systematic[0,0]
    ,cm_perfect_Systematic[1,1], cm_perfect_Systematic[0,0]
    ,cm_perfect_Systematic[1,1], cm_perfect_Systematic[0,0]
    ,cm_perfect_Systematic[1,1], cm_perfect_Systematic[0,0]
    ,cm_Systematic[0][1,1], cm_Systematic[0][1,0], cm_Systematic[0][0,0], cm_Systematic[0][0,1]
    ,cm_Systematic[1][1,1], cm_Systematic[1][1,0], cm_Systematic[1][0,0], cm_Systematic[1][0,1]
    ,cm_Systematic[2][1,1], cm_Systematic[2][1,0], cm_Systematic[2][0,0], cm_Systematic[2][0,1]
    ,cm_Systematic[3][1,1], cm_Systematic[3][1,0], cm_Systematic[3][0,0], cm_Systematic[3][0,1]],
    textinfo = "label+value+percent parent",
    branchvalues='total'
))

fig2.update_layout(margin = dict(t=25, l=25, r=25, b=25))
fig2.show()

In [ ]:
SVM_tuning = joblib.load('ML_Models/ML_SVM_tuning.pkl')
iforest_tuning = joblib.load('ML_Models/ML_iforest_tuning.pkl')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=np.arange(1,11), y=iforest_tuning.cv_results_['mean_test_F1'], text=np.round(iforest_tuning.cv_results_['mean_test_F1'],3)))
fig.update_layout(title='Mean F1-Score of the Different Cross-validation Tests for Each Combination of Hyperparameters',
                   yaxis_title='F1-Score',
                   xaxis=dict(
                    title='Combination Number',
                    tickmode='linear'))
fig.update_yaxes(range=[0.6, 0.99])


fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=np.arange(1,11), y=iforest_tuning.cv_results_['std_test_F1'], text=np.round(iforest_tuning.cv_results_['std_test_F1'],3)))
fig.update_layout(title='Standard Deviation of the F1-Score of the Different Cross-validation Tests<br>for Each Combination of Hyperparameters',
                   yaxis_title='Standard Deviation',
                   xaxis=dict(
                    title='Combination Number',
                    tickmode='linear'))
fig.update_yaxes(range=[0, 0.006])


fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=np.arange(1,11), y=iforest_tuning.cv_results_['mean_test_F1'], text=np.round(iforest_tuning.cv_results_['mean_test_F1'],3)))
fig.update_layout(title='Mean F1-Score of the Different Cross-validation Tests for Each Combination of Hyperparameters',
                   yaxis_title='F1-Score',
                   xaxis=dict(
                    title='Combination Number',
                    tickmode='linear'))
fig.update_yaxes(range=[0.6, 0.99])


fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=np.arange(1,43), y=SVM_tuning.cv_results_['mean_test_score'][0:42], text=np.round(SVM_tuning.cv_results_['mean_test_score'][0:42],3),
marker_color= 'blue'))
fig.update_layout(title='Mean F1-Score of the Different Cross-validation Tests for Each Combination of Hyperparameters<br>Iternation 1',
                   yaxis_title='F1-Score',
                   xaxis=dict(
                    title='Combination Number',
                    tickmode='linear'))
fig.update_yaxes(range=[0, 0.99])


fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=['11','40','33','39','15','36','18','31','26','32','38','25','22','29'], y=SVM_tuning.cv_results_['mean_test_score'][42:56], text=np.round(SVM_tuning.cv_results_['mean_test_score'][42:56],3),
marker_color='red'))
fig.update_layout(title='Mean F1-Score of the Different Cross-validation Tests for Each Combination of Hyperparameters<br>Iternation 2',
                   yaxis_title='F1-Score',
                   xaxis=dict(
                    title='Combination Number',
                    tickmode='linear'))
fig.update_yaxes(range=[0, 0.99])


fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=['22','25','38','32','29'], y=SVM_tuning.cv_results_['mean_test_score'][56:61], text=np.round(SVM_tuning.cv_results_['mean_test_score'][56:61],3),
marker_color='yellow'))
fig.update_layout(title='Mean F1-Score of the Different Cross-validation Tests for Each Combination of Hyperparameters<br>Iternation 3',
                   yaxis_title='F1-Score',
                   xaxis=dict(
                    title='Combination Number',
                    tickmode='linear'))
fig.update_yaxes(range=[0, 0.99])


fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=['29','32'], y=SVM_tuning.cv_results_['mean_test_score'][61:63], text=np.round(SVM_tuning.cv_results_['mean_test_score'][61:63],3),
marker_color='green'))
fig.update_layout(title='Mean F1-Score of the Different Cross-validation Tests for Each Combination of Hyperparameters<br>Iternation 4',
                   yaxis_title='F1-Score',
                   xaxis=dict(
                    title='Combination Number',
                    tickmode='linear'))
fig.update_yaxes(range=[0, 0.99])


fig.show()